# Experiments-Prepare

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [46]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.detection as detection 

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy
import os

In [108]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
reload(environment)
reload(detection)
reload(util)

<module 'source.util' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/util.py'>

## Generating Targets

In [72]:
targets_dict = {i:[] for i in range(2,13)}
T = 15 #np.random.randint(2,10)
time_horizon =1000
while sum([len(targets_dict[x]) for x in range(2,13)]) < 9:
    targets = [round(x,3) for x in util.gen_norm_targets(T)]
    values = tuple((v, v) for v in targets)
    g = game.Game(values, time_horizon)
    g.attackers = [1]
    g.defenders = [0]
    s = util.support(g)
    if len(targets_dict[len(s)])==0:
        targets_dict[len(s)].append(s)

## Writing batch file

In [110]:
experiment_folder = "../Experiments/Stackelberg_KnownStochastic"
time_horizon = 1000
defender = ["br_mab1-1", "br_expert1-1","b2bw2w1"] #,"holmes1-1"]
if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)
for T in range(2,11,2):
    if not os.path.exists(experiment_folder + "/" + str(T)):
        os.makedirs(experiment_folder + "/" + str(T))
    for P in range(1,4):
        targets, profiles = util.gen_setting(T, P, targets_dict, time_horizon)
        folder = experiment_folder + "/" + str(T) + "/" + str(P)
        if not os.path.exists(folder):
            os.makedirs(folder)
        batch_name = "_".join(defender)
        batch_file = batch_name + ".csv"
        batch_path = folder + "/" + batch_file
        with open(batch_path, "w+") as f:
            f.write(util.print_header(targets, profiles))
            for p in profiles: # modify profiles...
                for d in defender: 
                    f.write(util.print_row(targets, time_horizon,d,p))